This notebook will be dedicated towards:

1.) The yearly outbreak of malaria information from Sudan, South Sudan, Chad, and Ethiopia from 2000 -2022 (using 60.4 cases as the threshold for outbreak)
https://www.who.int/data/gho/data/themes/malaria 

2.) Gain NDVI and NDWI information from all four listed countries using Sentinel Data 
https://github.com/OmdenaAI/SudanChapter_AnalyzeHealthcareAccessibility/tree/main/04_Data_analysis/final_data_folder
https://github.com/OmdenaAI/SudanChapter_AnalyzeHealthcareAccessibility/tree/main/04_Data_analysis/Geographic/final_datasets

3.) Model the data and test it... with and without including country labels
--Logistic Regression
--Random Forest
--Neural Network

Imports

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta
import prophet

1.) Yearly outbreak information

In [6]:
df_mal = pd.read_csv('Desktop/OMDENA_FINAL_DATASETS/malaria_cases.csv')

In [8]:
df_mal.head()

,IND_ID,IND_CODE,IND_UUID,IND_PER_CODE,DIM_TIME,DIM_TIME_TYPE,DIM_GEO_CODE_M49,DIM_GEO_CODE_TYPE,DIM_PUBLISH_STATE_CODE,IND_NAME,GEO_NAME_SHORT,RATE_PER_1000_N,RATE_PER_1000_NL,RATE_PER_1000_NU
0,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2012,YEAR,9,REGION,PUBLISHED,Malaria cases,Oceania,148.12022,54.17984,293.38486
1,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2011,YEAR,15,REGION,PUBLISHED,Malaria cases,Northern Africa,7.06374,5.22736,9.32541
2,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2010,YEAR,18,REGION,PUBLISHED,Malaria cases,Southern Africa,1.53955,1.27314,1.98855
3,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2005,YEAR,31,COUNTRY,PUBLISHED,Malaria cases,Azerbaijan,1.21551,1.21551,1.21551
4,442CEA8MALARIA_EST_INCIDENCE,MALARIA_EST_INCIDENCE,442CEA8,MALARIA_EST_INCIDENCE,2012,YEAR,800,COUNTRY,PUBLISHED,Malaria cases,Uganda,376.55597,296.59672,471.17035


In [11]:
df_mal.shape

(3588, 14)

In [13]:
countries = ['Sudan','Chad','South Sudan','Ethiopia']

In [15]:
df_mal = df_mal[df_mal['GEO_NAME_SHORT'].isin(countries)]

In [17]:
df_mal.shape

(92, 14)

In [25]:
df_mal['DIM_TIME'].unique()

array([2020, 2005, 2007, 2009, 2013, 2010, 2011, 2018, 2006, 2016, 2002,
       2017, 2008, 2003, 2015, 2004, 2001, 2000, 2014, 2012, 2022, 2021,
       2019])

In [27]:
df_melt = pd.melt(df_mal, id_vars=['GEO_NAME_SHORT','DIM_TIME'], value_vars='RATE_PER_1000_N')

In [39]:
df_melt = df_melt.sort_values(['GEO_NAME_SHORT','DIM_TIME'])

In [51]:
df_melt = df_melt.rename(columns={'value':'Incidence_per_1000_people'})

In [53]:
df_melt.head()

,GEO_NAME_SHORT,DIM_TIME,Incidence_per_1000_people
0,Chad,2000,269.14078
1,Chad,2001,272.50860
2,Chad,2002,239.03563
3,Chad,2003,235.30286
4,Chad,2004,232.20829


In [43]:
df_melt = df_melt.drop(columns='variable')

In [45]:
df_melt = df_melt.reset_index(drop=True)

In [55]:
def outbreak_label(df):

    out_list = []

    for i in range(len(df)):
        if df.loc[i,'Incidence_per_1000_people'] >= 60.4:
            out_list.append(1)

        else:
            out_list.append(0)

    out_frame = pd.DataFrame({'outbreak':out_list})


    return out_frame

In [57]:
outbreak_frame = outbreak_label(df_melt)

In [61]:
outbreak_frame.head()

,outbreak
0,1
1,1
2,1
3,1
4,1


In [63]:
outbreak_df = pd.concat([df_melt, outbreak_frame], axis=1)

In [65]:
outbreak_df.head()

,GEO_NAME_SHORT,DIM_TIME,Incidence_per_1000_people,outbreak
0,Chad,2000,269.14078,1
1,Chad,2001,272.50860,1
2,Chad,2002,239.03563,1
3,Chad,2003,235.30286,1
4,Chad,2004,232.20829,1


In [67]:
outbreak_df.groupby('GEO_NAME_SHORT').sum()

,DIM_TIME,Incidence_per_1000_people,outbreak
GEO_NAME_SHORT,,,
Chad,46253,5211.45941,23
Ethiopia,46253,3726.08846,19
South Sudan,46253,6699.54248,23
Sudan,46253,1253.20170,8


Now we have the four countries and their outbreak labels

2.)Extract the NDVI and NDWI information from the three countries (South Sudan, Ethiopia, and Chad)... Then use existing code to get NDVI and NDWI values from Sudan

In [127]:
from prophet import Prophet

In [77]:
df_et = pd.read_csv('desktop/OMDENA_FINAL_DATASETS/eth_geographic_yearly.csv')

In [79]:
df_et.head()

,year,week,state,district,district_subdivision,ndvi_vegetation_index,ndvi_recording_exact_date,ndwi_water_content_index,ndwi_recording_exact_date
0,2015,43,Afar,AfarZone2,AbAla,0.002469,2015-10-20,-0.011014,2015-10-20
1,2015,43,Oromia,HoroGuduru,Ababo,0.002286,2015-10-20,-0.008143,2015-10-20
2,2015,43,Oromia,HoroGuduru,AbayChomen,0.002286,2015-10-20,-0.008143,2015-10-20
3,2015,43,Oromia,Borena,Abaya,0.002286,2015-10-20,-0.008143,2015-10-20
4,2015,43,Oromia,HoroGuduru,AbeDongoro,0.002286,2015-10-20,-0.008143,2015-10-20


In [87]:
df_ch = pd.read_csv('desktop/OMDENA_FINAL_DATASETS/tcd_geographic_yearly.csv')

In [89]:
df_ch.head()

,year,week,state,district,district_subdivision,ndvi_vegetation_index,ndvi_recording_exact_date,ndwi_water_content_index,ndwi_recording_exact_date
0,2015,43,Ouaddaï,DjourfAlAhmar,Abker,0.003521,2015-10-20,-0.009865,2015-10-20
1,2015,43,WadiFira,Biltine,Abou-Charib-I,0.003521,2015-10-20,-0.009865,2015-10-20
2,2015,43,WadiFira,Biltine,Abou-Charib-Ii,0.003521,2015-10-20,-0.009865,2015-10-20
3,2015,43,Hadjer-Lamis,HarazeAlBiar,Afrouk,0.002676,2015-10-20,-0.008848,2015-10-20
4,2015,43,Moyen-Chari,LacIro,Alako,0.000096,2015-10-20,-0.007119,2015-10-20


In [91]:
df_ssd = pd.read_csv('desktop/OMDENA_FINAL_DATASETS/ssd_geographic_yearly.csv')

In [93]:
df_ssd.head()

,year,week,state,district,district_subdivision,ndvi_vegetation_index,ndvi_recording_exact_date,ndwi_water_content_index,ndwi_recording_exact_date
0,2015,43,Jungoli,Ayod,Aeiud,0.00428,2015-10-20,-0.01043,2015-10-20
1,2015,43,Jungoli,NahrAtiem,Akoba,0.00428,2015-10-20,-0.01043,2015-10-20
2,2015,43,EasternEquatoria,Amatonge,Akotous,0.00428,2015-10-20,-0.01043,2015-10-20
3,2015,43,NorthBahr-al-Ghazal,Aweil,Aweil,0.00428,2015-10-20,-0.01043,2015-10-20
4,2015,43,UpperNile,Baleit,Baleit,0.00428,2015-10-20,-0.01043,2015-10-20


In [517]:
def ndvi_ndwi_df_extraction(df, state):

    #subselect for state that best represents the population, preferably the capital
    #subselect for ndvi and ndwi information
    #transform the columns and indices as appropriate

    state_sub = df[df['state'] == state]

    ndvi_sub = state_sub[['ndvi_recording_exact_date','ndvi_vegetation_index']]
    ndvi_sub = ndvi_sub.rename(columns={'ndvi_recording_exact_date':'ds', 'ndvi_vegetation_index':'y'})
    ndvi_sub = ndvi_sub.drop_duplicates(keep='first')
    ndvi_sub = ndvi_sub.reset_index(drop=True)


    ndwi_sub = state_sub[['ndwi_recording_exact_date', 'ndwi_water_content_index']]
    ndwi_sub = ndwi_sub.rename(columns={'ndwi_recording_exact_date':'ds', 'ndwi_water_content_index':'y'})
    ndwi_sub = ndwi_sub.drop_duplicates(keep='first')
    ndwi_sub = ndwi_sub.reset_index(drop=True)



    #create forecasters with the current data and then create dataframe with the forecasted information from 2000 on 

    
    m_v = Prophet()

    m_v.fit(ndvi_sub)


    m_w = Prophet()

    m_w.fit(ndwi_sub)
    
    
    start_dt = date(2000, 1, 1)
    end_dt = date(2015, 10, 19)

    # difference between current and previous date
    delta = timedelta(days=1)

    # store the dates between two dates in a list
    dates = []

    while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
        dates.append(start_dt.isoformat())
    # increment start date by timedelta
        start_dt += delta

    #create date dataframe

    past_dates = pd.DataFrame(data = {'ds':dates+ [x for x in ndvi_sub['ds']]})
    
    #create ndvi and ndwi values from fitted models
    forecast_past_v = m_v.predict(past_dates)

    forecast_past_v['ds'] = pd.to_datetime(forecast_past_v['ds'])

    forecast_past_v['year'] = [x.year for x in forecast_past_v.ds]

    forecast_past_v['month'] = [x.month for x in forecast_past_v.ds]

    forecast_past_v = forecast_past_v.groupby(['year','month']).mean()

    forecast_past_v = forecast_past_v.reset_index(drop=True)

    forecast_past_v['year'] = [x.year for x in forecast_past_v.ds]

    forecast_past_v['month'] = [x.month for x in forecast_past_v.ds]

    state_list = []

    for x in range(len(forecast_past_v)):
        state_list.append(state)

    state_df = pd.DataFrame({'state':state_list})

    forecast_past_v = pd.concat([forecast_past_v[['ds','yhat','month','year']], state_df], axis=1)

    forecast_past_v = forecast_past_v.rename(columns = {'yhat':'ndvi_value'})
    

    
    forecast_past_w = m_w.predict(past_dates)

    forecast_past_w['ds'] = pd.to_datetime(forecast_past_w['ds'])

    forecast_past_w['year'] = [x.year for x in forecast_past_w.ds]

    forecast_past_w['month'] = [x.month for x in forecast_past_w.ds]

    forecast_past_w = forecast_past_w.groupby(['year','month']).mean()

    forecast_past_w = forecast_past_w.reset_index(drop=True)

    forecast_past_w['year'] = [x.year for x in forecast_past_w.ds]

    forecast_past_w['month'] = [x.month for x in forecast_past_w.ds]

    forecast_past_w = pd.concat([forecast_past_w[['ds','yhat','month','year']], state_df], axis=1)

    forecast_past_w = forecast_past_w.rename(columns = {'yhat':'ndwi_value'})

    #merge ndwi and ndwi values
    

    forecast_df = pd.merge(left = forecast_past_v, right = forecast_past_w,how='left', left_on = 'ds', right_on = 'ds')


    return forecast_df

Lets use the capitals of Ethiopia (AddisAbeba), Chad (N'Djamena), and South Sudan (Juba -- in Central Equatoria state)

In [105]:
df_et.state.unique()

array(['Afar', 'Oromia', 'Amhara', 'SouthernNations,Nationalities',
       'GambelaPeoples', 'Somali', 'AddisAbeba', 'Tigray',
       'Benshangul-Gumaz', 'DireDawa', 'HarariPeople'], dtype=object)

In [109]:
df_ssd.state.unique()

array(['Jungoli', 'EasternEquatoria', 'NorthBahr-al-Ghazal', 'UpperNile',
       'Unity', 'WestEquatoria', 'Warap', 'CentralEquatoria', 'Lakes',
       'WestBahr-al-Ghazal'], dtype=object)

In [111]:
df_ch.state.unique()

array(['Ouaddaï', 'WadiFira', 'Hadjer-Lamis', 'Moyen-Chari', 'Salamat',
       'Kanem', 'LogoneOriental', 'Chari-Baguirmi', 'Tandjilé',
       'LogoneOccidental', 'Sila', 'Mandoul', 'Mayo-KebbiEst', 'Guéra',
       'Mayo-KebbiOuest', 'Lac', 'Borkou', 'Batha', 'EnnediOuest',
       'BarhelGhazel', "VilledeN'Djamena", 'Tibesti'], dtype=object)

In [519]:
et_res =  ndvi_ndwi_df_extraction(df_et, 'AddisAbeba')

17:15:34 - cmdstanpy - INFO - Chain [1] start processing
17:15:34 - cmdstanpy - INFO - Chain [1] done processing
17:15:34 - cmdstanpy - INFO - Chain [1] start processing
17:15:34 - cmdstanpy - INFO - Chain [1] done processing


In [521]:
et_res.shape

(301, 9)

In [523]:
et_res.head()

,ds,ndvi_value,month_x,year_x,state_x,ndwi_value,month_y,year_y,state_y
0,2000-01-16 00:00:00,0.166194,1,2000,AddisAbeba,-0.132151,1,2000,AddisAbeba
1,2000-02-15 00:00:00,0.179835,2,2000,AddisAbeba,-0.144038,2,2000,AddisAbeba
2,2000-03-16 00:00:00,0.207015,3,2000,AddisAbeba,-0.163345,3,2000,AddisAbeba
3,2000-04-15 12:00:00,0.234121,4,2000,AddisAbeba,-0.187084,4,2000,AddisAbeba
4,2000-05-16 00:00:00,0.187948,5,2000,AddisAbeba,-0.150527,5,2000,AddisAbeba


In [525]:
ssd_res =  ndvi_ndwi_df_extraction(df_ssd, 'CentralEquatoria')

17:15:39 - cmdstanpy - INFO - Chain [1] start processing
17:15:39 - cmdstanpy - INFO - Chain [1] done processing
17:15:39 - cmdstanpy - INFO - Chain [1] start processing
17:15:39 - cmdstanpy - INFO - Chain [1] done processing


In [527]:
ssd_res.head()

,ds,ndvi_value,month_x,year_x,state_x,ndwi_value,month_y,year_y,state_y
0,2000-01-16 00:00:00,0.148613,1,2000,CentralEquatoria,-0.115094,1,2000,CentralEquatoria
1,2000-02-15 00:00:00,0.160969,2,2000,CentralEquatoria,-0.126214,2,2000,CentralEquatoria
2,2000-03-16 00:00:00,0.191681,3,2000,CentralEquatoria,-0.150353,3,2000,CentralEquatoria
3,2000-04-15 12:00:00,0.214516,4,2000,CentralEquatoria,-0.168998,4,2000,CentralEquatoria
4,2000-05-16 00:00:00,0.168091,5,2000,CentralEquatoria,-0.131789,5,2000,CentralEquatoria


In [529]:
ch_res = ndvi_ndwi_df_extraction(df_ch, "VilledeN'Djamena")

17:15:41 - cmdstanpy - INFO - Chain [1] start processing
17:15:41 - cmdstanpy - INFO - Chain [1] done processing
17:15:41 - cmdstanpy - INFO - Chain [1] start processing
17:15:41 - cmdstanpy - INFO - Chain [1] done processing


In [531]:
ch_res.head()

,ds,ndvi_value,month_x,year_x,state_x,ndwi_value,month_y,year_y,state_y
0,2000-01-16 00:00:00,0.151570,1,2000,VilledeN'Djamena,-0.121412,1,2000,VilledeN'Djamena
1,2000-02-15 00:00:00,0.165587,2,2000,VilledeN'Djamena,-0.132762,2,2000,VilledeN'Djamena
2,2000-03-16 00:00:00,0.183248,3,2000,VilledeN'Djamena,-0.148817,3,2000,VilledeN'Djamena
3,2000-04-15 12:00:00,0.212395,4,2000,VilledeN'Djamena,-0.169137,4,2000,VilledeN'Djamena
4,2000-05-16 00:00:00,0.159230,5,2000,VilledeN'Djamena,-0.122242,5,2000,VilledeN'Djamena


In [533]:
ch_res.shape

(301, 9)

lets do Khartoum from Sudan now

In [192]:
ndvi_df_sd = pd.read_csv('Desktop/OMDENA_FINAL_DATASETS/final_ndvi_geographic.csv')

In [196]:
ndvi_df_sd.head()

,date,value,state
0,2015-10-20,0.005310,AlJazirah
1,2015-11-29,0.017227,AlJazirah
2,2015-12-09,0.058856,AlJazirah
3,2015-12-19,0.030519,AlJazirah
4,2015-12-29,0.084125,AlJazirah


In [194]:
ndwi_df_sd = pd.read_csv('Desktop/OMDENA_FINAL_DATASETS/final_ndwi_geographic.csv')

In [198]:
ndwi_df_sd.head()

,date,value,state
0,2015-10-20,-0.011545,AlJazirah
1,2015-11-29,-0.040840,AlJazirah
2,2015-12-09,-0.094806,AlJazirah
3,2015-12-19,0.015846,AlJazirah
4,2015-12-29,-0.074044,AlJazirah


In [338]:
def ndvi_ndwi_df_sudan(df_ndvi, df_ndwi, state):

    #subselect for state that best represents the population, preferably the capital
    #subselect for ndvi and ndwi information
    #transform the columns and indices as appropriate

    state_sub1 = df_ndvi[df_ndvi['state'] == state]
    state_sub2 = df_ndwi[df_ndwi['state'] == state]

    ndvi_sub = state_sub1[['date','value']]
    ndvi_sub = ndvi_sub.rename(columns={'date':'ds', 'value':'y'})
    ndvi_sub = ndvi_sub.drop_duplicates(keep='first')
    ndvi_sub = ndvi_sub.reset_index(drop=True)


    ndwi_sub = state_sub2[['date','value']]
    ndwi_sub = ndwi_sub.rename(columns={'date':'ds', 'value':'y'})
    ndwi_sub = ndwi_sub.drop_duplicates(keep='first')
    ndwi_sub = ndwi_sub.reset_index(drop=True)



    #create forecasters with the current data and then create dataframe with the forecasted information from 2000 onto present 

    
    m_v = Prophet()

    m_v.fit(ndvi_sub)


    m_w = Prophet()

    m_w.fit(ndwi_sub)

    #create dates
    
    start_dt = date(2000, 1, 1)
    end_dt = date(2015, 10, 19)

    # difference between current and previous date
    delta = timedelta(days=1)

    # store the dates between two dates in a list
    dates = []

    while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
        dates.append(start_dt.isoformat())
    # increment start date by timedelta
        start_dt += delta

    #create date dataframe

    past_dates = pd.DataFrame(data = {'ds':dates+ [x for x in ndvi_sub['ds']]})

    #now predict ndvi values from the dataframe
    
    forecast_past_v = m_v.predict(past_dates)

    forecast_past_v['ds'] = pd.to_datetime(forecast_past_v['ds'])

    forecast_past_v['year'] = [x.year for x in forecast_past_v.ds]

    forecast_past_v['month'] = [x.month for x in forecast_past_v.ds]

    forecast_past_v = forecast_past_v.groupby(['year','month']).mean()

    forecast_past_v = forecast_past_v.reset_index(drop=True)

    forecast_past_v['year'] = [x.year for x in forecast_past_v.ds]

    forecast_past_v['month'] = [x.month for x in forecast_past_v.ds]
    

    #briefly generate a list of intended states to concat with the dataframe
    state_list = []

    for x in range(len(forecast_past_v)):
        state_list.append(state)

    state_df = pd.DataFrame({'state':state_list})
    

    forecast_past_v = pd.concat([forecast_past_v[['ds','yhat','month','year']], state_df], axis=1)

    forecast_past_v = forecast_past_v.rename(columns = {'yhat':'ndvi_value'})
    
    
    #now predict the ndwi values
    
    forecast_past_w = m_w.predict(past_dates)

    forecast_past_w['ds'] = pd.to_datetime(forecast_past_w['ds'])

    forecast_past_w['year'] = [x.year for x in forecast_past_w.ds]

    forecast_past_w['month'] = [x.month for x in forecast_past_w.ds]

    forecast_past_w = forecast_past_w.groupby(['year','month']).mean()

    forecast_past_w = forecast_past_w.reset_index(drop=True)

    forecast_past_w['year'] = [x.year for x in forecast_past_w.ds]

    forecast_past_w['month'] = [x.month for x in forecast_past_w.ds]

    forecast_past_w = pd.concat([forecast_past_w[['ds','yhat','month','year']], state_df], axis=1)

    forecast_past_w = forecast_past_w.rename(columns = {'yhat':'ndwi_value'})
    

    #merge ndvi and ndwi information together and return the dataframe
    forecast_df = pd.merge(left = forecast_past_v, right = forecast_past_w, left_on = 'ds', right_on = 'ds')


    return forecast_df

In [340]:
sd_res = ndvi_ndwi_df_sudan(ndvi_df_sd, ndwi_df_sd, 'Khartoum')

14:06:41 - cmdstanpy - INFO - Chain [1] start processing
14:06:41 - cmdstanpy - INFO - Chain [1] done processing
14:06:41 - cmdstanpy - INFO - Chain [1] start processing
14:06:41 - cmdstanpy - INFO - Chain [1] done processing


In [342]:
sd_res.head()

,ds,ndvi_value,month_x,year_x,state_x,ndwi_value,month_y,year_y,state_y
0,2000-01-16 00:00:00,0.172548,1,2000,Khartoum,-0.121937,1,2000,Khartoum
1,2000-02-15 00:00:00,0.190022,2,2000,Khartoum,-0.136662,2,2000,Khartoum
2,2000-03-16 00:00:00,0.219407,3,2000,Khartoum,-0.159775,3,2000,Khartoum
3,2000-04-15 12:00:00,0.235572,4,2000,Khartoum,-0.174150,4,2000,Khartoum
4,2000-05-16 00:00:00,0.188766,5,2000,Khartoum,-0.136522,5,2000,Khartoum


In [344]:
sd_res.shape

(300, 9)

Lets merge the malaria results with their corresponding states and years

In [356]:
outbreak_df_sd = outbreak_df[outbreak_df['GEO_NAME_SHORT'] == 'Sudan']

In [358]:
sd_merge = pd.merge(left = sd_res, right= outbreak_df_sd, how = 'left', left_on = 'year_x', right_on = 'DIM_TIME')

In [360]:
sd_merge.shape

(300, 13)

In [362]:
sd_merge.head()

,ds,ndvi_value,month_x,year_x,state_x,ndwi_value,month_y,year_y,state_y,GEO_NAME_SHORT,DIM_TIME,Incidence_per_1000_people,outbreak
0,2000-01-16 00:00:00,0.172548,1,2000,Khartoum,-0.121937,1,2000,Khartoum,Sudan,2000.0,92.77145,1.0
1,2000-02-15 00:00:00,0.190022,2,2000,Khartoum,-0.136662,2,2000,Khartoum,Sudan,2000.0,92.77145,1.0
2,2000-03-16 00:00:00,0.219407,3,2000,Khartoum,-0.159775,3,2000,Khartoum,Sudan,2000.0,92.77145,1.0
3,2000-04-15 12:00:00,0.235572,4,2000,Khartoum,-0.174150,4,2000,Khartoum,Sudan,2000.0,92.77145,1.0
4,2000-05-16 00:00:00,0.188766,5,2000,Khartoum,-0.136522,5,2000,Khartoum,Sudan,2000.0,92.77145,1.0


In [364]:
outbreak_df_et = outbreak_df[outbreak_df['GEO_NAME_SHORT'] == 'Ethiopia']

In [366]:
et_merge = pd.merge(left = et_res, right= outbreak_df_et, how = 'left', left_on = 'year_x', right_on = 'DIM_TIME')

In [368]:
et_merge.shape

(301, 13)

In [370]:
outbreak_df_ssd = outbreak_df[outbreak_df['GEO_NAME_SHORT'] == 'South Sudan']

In [372]:
ssd_merge = pd.merge(left = ssd_res, right= outbreak_df_ssd, how = 'left', left_on = 'year_x', right_on = 'DIM_TIME')

In [374]:
outbreak_df_ch = outbreak_df[outbreak_df['GEO_NAME_SHORT'] == 'Chad']

In [376]:
ch_merge = pd.merge(left = ch_res, right= outbreak_df_ch, how = 'left', left_on = 'year_x', right_on = 'DIM_TIME')

In [378]:
final_df = pd.concat([sd_merge, et_merge], axis=0)

In [380]:
final_df = pd.concat([final_df, ssd_merge], axis=0)

In [382]:
final_df = pd.concat([final_df, ch_merge], axis=0)

Lets us do a sanity check on the final df

In [385]:
final_df.head()

,ds,ndvi_value,month_x,year_x,state_x,ndwi_value,month_y,year_y,state_y,GEO_NAME_SHORT,DIM_TIME,Incidence_per_1000_people,outbreak
0,2000-01-16 00:00:00,0.172548,1,2000,Khartoum,-0.121937,1,2000,Khartoum,Sudan,2000.0,92.77145,1.0
1,2000-02-15 00:00:00,0.190022,2,2000,Khartoum,-0.136662,2,2000,Khartoum,Sudan,2000.0,92.77145,1.0
2,2000-03-16 00:00:00,0.219407,3,2000,Khartoum,-0.159775,3,2000,Khartoum,Sudan,2000.0,92.77145,1.0
3,2000-04-15 12:00:00,0.235572,4,2000,Khartoum,-0.174150,4,2000,Khartoum,Sudan,2000.0,92.77145,1.0
4,2000-05-16 00:00:00,0.188766,5,2000,Khartoum,-0.136522,5,2000,Khartoum,Sudan,2000.0,92.77145,1.0


In [387]:
final_df.shape

(1203, 13)

In [396]:
final_df.isna().sum()

ds                            0
ndvi_value                    0
month_x                       0
year_x                        0
state_x                       0
ndwi_value                    0
month_y                       0
year_y                        0
state_y                       0
GEO_NAME_SHORT               99
DIM_TIME                     99
Incidence_per_1000_people    99
outbreak                     99
dtype: int64

In [398]:
final_df = final_df.dropna()

In [400]:
final_df.shape

(1104, 13)

3.) Modeling
a.) without country information (inputs -- > ndvi, ndwi, month | outputs-- > outbreak prediction)
b.) with country information (inputs -- > ndvi, ndwi, month, country | outputs-- > outbreak prediction)

In [390]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

In [402]:
X = final_df[['ndvi_value','ndwi_value','month_x']]
y = final_df['outbreak']

In [404]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .2, stratify=y)

In [416]:
params = [{'solver':['liblinear','newton-cg'],'max_iter':[100, 200, 300, 500, 1000],'C':[1,10,100, 1000, 10000, 100000]}]

In [418]:
classifier = GridSearchCV(LogisticRegression(),
                      param_grid=params,
                      scoring='f1',
                      cv=5)

In [420]:
classifier.fit(Xtrain, ytrain)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [1, 10, 100, 1000, 10000, 100000],
                          'max_iter': [100, 200, 300, 500, 1000],
                          'solver': ['liblinear', 'newton-cg']}],
             scoring='f1')

In [422]:
accuracy_score(classifier.predict(Xtest), ytest)

0.7918552036199095

In [424]:
print("Classification Report for Test Data")
print(classification_report(ytest, classifier.predict(Xtest)))

Classification Report for Test Data
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        46
         1.0       0.79      1.00      0.88       175

    accuracy                           0.79       221
   macro avg       0.40      0.50      0.44       221
weighted avg       0.63      0.79      0.70       221



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


This model is skewed now towards the yes class (1) as opposed to the earlier datasets skew towards no class (0)

Random Forest Classifier

In [426]:
from sklearn.ensemble import RandomForestClassifier

In [428]:
grid_params = {'n_estimators':[50, 100, 200, 300, 500], 'criterion':['gini','entropy','log_loss'] }

gscv_rfc = GridSearchCV(RandomForestClassifier(), param_grid=grid_params, cv=5, scoring='f1')

rfc = gscv_rfc.fit(Xtrain, ytrain)

print (rfc.best_params_)
print (rfc.best_score_)

{'criterion': 'log_loss', 'n_estimators': 100}
0.9170078182869348


In [430]:
print("Classification Report for Test Data")
print(classification_report(ytest, rfc.predict(Xtest)))

Classification Report for Test Data
              precision    recall  f1-score   support

         0.0       0.72      0.46      0.56        46
         1.0       0.87      0.95      0.91       175

    accuracy                           0.85       221
   macro avg       0.80      0.71      0.74       221
weighted avg       0.84      0.85      0.84       221



Now lets repeat this with the countries added as one hot encoded information

In [453]:
one_hots = pd.get_dummies(final_df['GEO_NAME_SHORT'])

In [455]:
one_hots.head()

,Chad,Ethiopia,South Sudan,Sudan
0,False,False,False,True
1,False,False,False,True
2,False,False,False,True
3,False,False,False,True
4,False,False,False,True


In [457]:
final_df_dummies = pd.concat([final_df, one_hots], axis=1)

In [459]:
X = final_df_dummies[['ndvi_value','ndwi_value','month_x','Chad','Ethiopia','South Sudan','Sudan']]

In [461]:
y = final_df_dummies['outbreak']

In [487]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .2, stratify=y)

In [465]:
params = [{'solver':['liblinear','newton-cg'],'max_iter':[100, 200, 300, 500, 1000],'C':[1,10,100, 1000, 10000, 100000]}]

In [467]:
classifier = GridSearchCV(LogisticRegression(),
                      param_grid=params,
                      scoring='f1',
                      cv=5)

In [469]:
classifier.fit(Xtrain, ytrain)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': [1, 10, 100, 1000, 10000, 100000],
                          'max_iter': [100, 200, 300, 500, 1000],
                          'solver': ['liblinear', 'newton-cg']}],
             scoring='f1')

In [538]:
accuracy_score(classifier.predict(Xtest), ytest)

0.8778280542986425

In [473]:
print("Classification Report for Test Data")
print(classification_report(ytest, classifier.predict(Xtest)))

Classification Report for Test Data
              precision    recall  f1-score   support

         0.0       0.67      0.76      0.71        46
         1.0       0.93      0.90      0.92       175

    accuracy                           0.87       221
   macro avg       0.80      0.83      0.82       221
weighted avg       0.88      0.87      0.88       221



Using random forest

In [476]:
grid_params = {'n_estimators':[50, 100, 200, 300, 500], 'criterion':['gini','entropy','log_loss'] }

gscv_rfc = GridSearchCV(RandomForestClassifier(), param_grid=grid_params, cv=5, scoring='f1')

rfc = gscv_rfc.fit(Xtrain, ytrain)

print (rfc.best_params_)
print (rfc.best_score_)

{'criterion': 'log_loss', 'n_estimators': 200}
0.9402910954012581


In [478]:
print("Classification Report for Test Data")
print(classification_report(ytest, rfc.predict(Xtest)))

Classification Report for Test Data
              precision    recall  f1-score   support

         0.0       0.73      0.59      0.65        46
         1.0       0.90      0.94      0.92       175

    accuracy                           0.87       221
   macro avg       0.81      0.76      0.78       221
weighted avg       0.86      0.87      0.86       221



Neural Network (100 parameters max since 1000 datapoints)

In [483]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

In [515]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(Xtrain)
X_test = scaler.transform(Xtest)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(ytrain.to_numpy(), dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(ytest.to_numpy(), dtype=torch.float32).unsqueeze(1)

# Define a neural network with at most 110 parameters (a tenth of the observations)
class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.fc1 = nn.Linear(7, 12)  
        self.fc2 = nn.Linear(12, 1)  
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Check parameter count
model = SmallNet()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}")

# Ensure the number of parameters is within the limit
assert num_params <= 110, "Model exceeds 100 parameters!"

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluate the model
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_class = (y_pred > 0.5).float()
    accuracy = (y_pred_class == y_test_tensor).float().mean()
    print(f"Test Accuracy: {accuracy:.4f}")

Number of parameters: 109
Epoch [10/100], Loss: 0.5316
Epoch [20/100], Loss: 0.4531
Epoch [30/100], Loss: 0.4045
Epoch [40/100], Loss: 0.3718
Epoch [50/100], Loss: 0.3489
Epoch [60/100], Loss: 0.3324
Epoch [70/100], Loss: 0.3204
Epoch [80/100], Loss: 0.3118
Epoch [90/100], Loss: 0.3056
Epoch [100/100], Loss: 0.3009
Test Accuracy: 0.8778


All three models are close to 88% for accuracy, with the logistic regression having the best f1 score (.88).  The logistic regression wiht country information should move onto the modeling deployment stage.